In [1]:
import torch
import sqlite3
import gradio as gr

/Users/odai/5588-hands-on-20240912/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the pre-trained YOLOv5 model
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

Using cache found in /Users/odai/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-12 Python-3.11.6 torch-2.4.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
conn = sqlite3.connect("hotdog_predictions.db")
c = conn.cursor()
c.execute(
    """
    CREATE TABLE IF NOT EXISTS predictions (
        id INTEGER PRIMARY KEY,
        image_path TEXT,
        prediction TEXT,
        confidence REAL
    )
"""
)
conn.commit()
conn.close()

In [4]:
def predict_and_store(image, image_path):
    results = model(image)
    results = results.pandas().xyxy[0]  # Results as DataFrame

    # Check for hot dogs in the predictions
    hotdog_detected = any(results["name"].str.contains("hot dog"))
    prediction = "hot dog" if hotdog_detected else "not a hot dog"
    confidence = (
        results[results["name"] == "hot dog"]["confidence"].max()
        if hotdog_detected
        else 0
    )

    # Store in database
    conn = sqlite3.connect("hotdog_predictions.db")
    c = conn.cursor()
    c.execute(
        "INSERT INTO predictions (image_path, prediction, confidence) VALUES (?, ?, ?)",
        (image_path, prediction, confidence),
    )
    conn.commit()
    conn.close()

    return prediction, float(confidence)

In [5]:
def classify_image(image):
    image_path = "image.jpg"
    # image.save(image_path)
    image = torch.from_numpy(image).float().permute(2, 0, 1) / 255.0
    image = image.unsqueeze(0)

    prediction, confidence = predict_and_store(image, image_path)
    return prediction

interface = gr.Interface(fn=classify_image, inputs="image", outputs="text")
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://55905d9ff0206f6c5d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/odai/5588-hands-on-20240912/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/odai/5588-hands-on-20240912/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/odai/5588-hands-on-20240912/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/odai/5588-hands-on-20240912/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/odai/5588-hands-on-20240912/.venv/l